<span style="color:#8B949E;">
<b>Note de lecture</b> — Notebook issu de tests itératifs (“speed-tests”).  
Le corpus utilisé ici est un <b>dataset de substitution</b> (non aligné client) uniquement pour valider la mécanique (retrieval + évaluation) et dérouler la roadmap.  
Pour le chemin complet, suivre l’ordre 01 → 10 et lire en priorité les sections Markdown.
</span>


# 🧪 Stage 3 — BM25 avec filtrage métier minimal (Code du travail)

Ce notebook présente et exécute le script **`3_Benchmark_filtre-BM25.py`** sous forme *pas-à-pas*, avec une lecture claire :

- 🧱 construction de l’index **BM25** sur un corpus **filtré**
- 🧾 définition d’un mini **benchmark** (questions / mots-clés)
- 📏 calcul de **Recall@k**, **MRR**, **nDCG@k**
- 🔍 **analyse qualitative** des résultats (verbatim)

> Astuce compatibilité : ce notebook est prévu pour fonctionner en Python **3.8+**.


## ✅ Pré-requis & environnement

### 📦 Dépendances
- `rank_bm25`
- un module local **`corpus_loader.py`** exposant `documents` (liste de dicts avec au moins `{"text": ...}`)

### 🧭 Vérification rapide


In [7]:
# Vérifie la version Python (utile pour éviter les soucis de type hints en notebook)
import sys
print(sys.version)


3.9.13 (tags/v3.9.13:6de2ca5, May 17 2022, 16:36:42) [MSC v.1929 64 bit (AMD64)]


In [8]:
# Installation (si besoin) — à exécuter une seule fois
# !pip install rank_bm25


## 🗒️ Contexte (extrait)

> « Après la baseline BM25 brute, j’ai introduit un filtrage métier minimal (par code) 
> pour réduire le bruit avant toute couche sémantique, puis j’ai re-mesuré 
> Recall@k, MRR et nDCG afin de quantifier l’impact. »
> 
> « Les métriques me donnent une vision globale, mais je complète toujours par 
> une analyse qualitative des résultats (VERBATIM), afin d’identifier précisément la nature 
> des faux positifs et des faux négatifs. »
> 
> « J’ai factorisé le chargement et le filtrage du corpus dans un module dédié afin de 
> garantir que toutes les expériences de retrieval reposent sur exactement le même périmètre documentaire. 
> Cela me permet de comparer BM25, dense et hybride sans biais expérimental. »
> 
> ===========================================================
> STAGE 3 – BM25 AVEC FILTRAGE MÉTIER MINIMAL
> ===========================================================
> 
> Objectif :
> Comparer la baseline BM25 brute avec une version intégrant
> un filtrage métier simple, avant toute couche sémantique.
> 
> Dans ce stage :
> - le corpus est restreint à un périmètre juridique pertinent
>   (ici : Code du travail),
> - l’index BM25 est reconstruit sur ce sous-ensemble,
> - le même benchmark (questions, métriques, k) est rejoué
>   afin de mesurer objectivement l’impact du filtrage.
> 
> Enjeux :
> - réduire le bruit (faux positifs),
> - améliorer la qualité du ranking,
> - observer l’évolution du Recall@k, de la MRR et de la nDCG,
> - préparer le terrain avant l’introduction d’embeddings.
> 
> Important :
> - aucune génération par LLM n’est utilisée,
> - aucune modification du benchmark n’est effectuée,
> - toute amélioration doit être justifiée par des métriques.

## 🧱 1) Chargement du corpus filtré & construction de l’index BM25

In [9]:
import os
import xml.etree.ElementTree as ET
from rank_bm25 import BM25Okapi

from corpus_loader import documents  # charge les documents filtré par corpus_loader.py
# pi

# - construction de L'index BM25 

def tokenize(text):
    """
    Tokenisation volontairement simple.
    Suffisante pour observer les limites du lexical pur.
    """
    return text.lower().split()

# Corpus tokenisé dans le MÊME ordre que documents
corpus_tokens = [tokenize(doc["text"]) for doc in documents]

# Construction de l'index BM25
bm25 = BM25Okapi(corpus_tokens)

print("Index BM25 construit")

Index BM25 construit


## 🎯 2) Jeu de questions (benchmark)

In [10]:
# 1. JEU DE QUESTIONS DU BENCHMARK
# =========================================================

"""
Chaque question est associée à :
- une formulation utilisateur réaliste
- une liste de documents juridiquement pertinents attendus

Dans un vrai projet :
- ces annotations seraient réalisées par des juristes
- ici, elles sont volontairement simples pour un POC
"""

benchmark_queries = [
    {
        "query_id": "Q1",
        "question": "Dans quels cas un CDI peut-il être rompu sans préavis ?",
        "relevant_keywords": [
            "faute grave",
            "rupture sans préavis",
            "L1234"
        ]
    },
    {
        "query_id": "Q2",
        "question": "Qu'est-ce qu'un licenciement pour motif économique ?",
        "relevant_keywords": [
            "motif économique",
            "L1233-3"
        ]
    },
    {
        "query_id": "Q3",
        "question": "Un salarié peut-il contester un licenciement ?",
        "relevant_keywords": [
            "contestation",
            "sanctions",
            "irrégularités du licenciement"
        ]
    }
]

# Remarque :
# Pour simplifier, on n'utilise pas encore de doc_id exact.
# On considère qu'un document est pertinent s'il contient
# un ou plusieurs mots-clés juridiques attendus.


# =========================================================

## 🔎 3) Recherche BM25 (ranking)

In [ ]:
# 2. FONCTION DE RECHERCHE BM25
# =========================================================

def bm25_search(query, documents, bm25, top_k=10):
    """
    Exécute une recherche BM25 sur le corpus.

    Paramètres :
    - query : question utilisateur (str)
    - documents : liste des documents indexés
    - bm25 : index BM25 déjà construit
    - top_k : nombre de résultats retournés

    Retour :
    - liste de documents classés par score décroissant
    """
    query_tokens = tokenize(query)
    scores = bm25.get_scores(query_tokens)

    ranked_docs = sorted(
        zip(documents, scores),
        key=lambda x: x[1],
        reverse=True
    )

    return ranked_docs[:top_k]


## 📏 4) Métriques (Recall@k, MRR, nDCG@k)

In [ ]:
# =========================================================
# 3. MÉTRIQUES DE BENCHMARK
# =========================================================

import math


def is_relevant(document_text, relevant_keywords):
    """
    Détermine si un document est pertinent pour une question.

    Ici :
    - un document est pertinent s'il contient au moins
      un mot-clé juridique attendu.
    """
    text_lower = document_text.lower()
    return any(keyword.lower() in text_lower for keyword in relevant_keywords)


def recall_at_k(results, relevant_keywords, k):
    """
    Recall@k :
    - 1 si au moins un document pertinent est présent dans le top-k
    - 0 sinon
    """
    top_k_docs = results[:k]
    return int(
        any(is_relevant(doc["text"], relevant_keywords) for doc, _ in top_k_docs)
    )


def reciprocal_rank(results, relevant_keywords):
    """
    MRR :
    - 1 / rang du premier document pertinent
    - 0 si aucun document pertinent n'est trouvé
    """
    for rank, (doc, _) in enumerate(results, start=1):
        if is_relevant(doc["text"], relevant_keywords):
            return 1 / rank
    return 0


def ndcg_at_k(results, relevant_keywords, k):
    """
    nDCG@k :
    - mesure la qualité globale du ranking
    - pénalise les documents pertinents mal classés
    """
    def dcg(res):
        score = 0.0
        for i, (doc, _) in enumerate(res[:k]):
            if is_relevant(doc["text"], relevant_keywords):
                score += 1 / math.log2(i + 2)
        return score

    actual_dcg = dcg(results)

    # DCG idéal : tous les documents pertinents en tête
    ideal_relevance = [1] * sum(
        is_relevant(doc["text"], relevant_keywords)
        for doc, _ in results[:k]
    )

    ideal_dcg = sum(
        1 / math.log2(i + 2)
        for i in range(len(ideal_relevance))
    )

    return actual_dcg / ideal_dcg if ideal_dcg > 0 else 0


## 🧮 5) Évaluation globale

In [13]:
# =========================================================
# 4. ÉVALUATION GLOBALE DU BENCHMARK
# =========================================================

def evaluate_benchmark(queries, documents, bm25, k=10):
    """
    Lance le benchmark sur l'ensemble des questions
    et calcule les métriques moyennes.
    """
    recall_scores = []
    mrr_scores = []
    ndcg_scores = []

    for q in queries:
        results = bm25_search(
            q["question"],
            documents,
            bm25,
            top_k=k
        )

        recall_scores.append(
            recall_at_k(results, q["relevant_keywords"], k)
        )
        mrr_scores.append(
            reciprocal_rank(results, q["relevant_keywords"])
        )
        ndcg_scores.append(
            ndcg_at_k(results, q["relevant_keywords"], k)
        )

    return {
        "Recall@k": sum(recall_scores) / len(recall_scores),
        "MRR": sum(mrr_scores) / len(mrr_scores),
        "nDCG@k": sum(ndcg_scores) / len(ndcg_scores)
    }


# =========================================================

# 5. LANCEMENT DU BENCHMARK
# =========================================================

results = evaluate_benchmark(
    benchmark_queries,
    documents,
    bm25,
    k=10
)

print("\n=== RÉSULTATS DU BENCHMARK BM25 (FILTRÉ Code du travail) ===")
for metric, value in results.items():
    print(f"{metric} : {value:.3f}")

"""
Lecture attendue des résultats :

- Recall@k faible :
  → présence de faux négatifs critiques
- MRR moyenne :
  → documents pertinents trouvés mais mal classés
- nDCG modérée :
  → ranking global perfectible

Ces résultats justifient :
- l'ajout de filtres métier
- puis d'une couche sémantique (embeddings)
"""

# =========================================================


=== RÉSULTATS DU BENCHMARK BM25 (FILTRÉ Code du travail) ===
Recall@k : 0.667
MRR : 0.375
nDCG@k : 0.438


"\nLecture attendue des résultats :\n\n- Recall@k faible :\n  → présence de faux négatifs critiques\n- MRR moyenne :\n  → documents pertinents trouvés mais mal classés\n- nDCG modérée :\n  → ranking global perfectible\n\nCes résultats justifient :\n- l'ajout de filtres métier\n- puis d'une couche sémantique (embeddings)\n"

## 🗣️ 6) Analyse qualitative (verbatim)

In [14]:
# 6. ANALYSE QUALITATIVE PAR QUESTION (VERBATIM)
# =========================================================

def display_results_per_query(queries, documents, bm25, k=5):
    """
    Affiche les résultats BM25 pour chaque question du benchmark,
    afin de permettre une analyse qualitative (verbatim).

    Objectif :
    - comprendre les faux positifs / faux négatifs,
    - analyser le ranking,
    - compléter la lecture des métriques chiffrées.
    """
    for q in queries:
        print("\n" + "=" * 80)
        print(f"Question : {q['question']}")
        print("=" * 80)

        results = bm25_search(
            q["question"],
            documents,
            bm25,
            top_k=k
        )

        for rank, (doc, score) in enumerate(results, start=1):
            relevant = is_relevant(doc["text"], q["relevant_keywords"])
            flag = "PERTINENT" if relevant else "NON PERTINENT"

            print(f"\n--- Rang {rank} | Score BM25 : {score:.2f} | {flag}")
            print(doc["text"][:400])



display_results_per_query(
    benchmark_queries,
    documents,
    bm25,
    k=10
)





Question : Dans quels cas un CDI peut-il être rompu sans préavis ?

--- Rang 1 | Score BM25 : 18.41 | NON PERTINENT
   LEGIARTI000021925677 LEGI article/LEGI/ARTI/00/00/21/92/56/LEGIARTI000021925677.xml Article   L3142-17 MODIFIE 2010-03-04 2016-08-10 AUTONOME   Code du travail  Partie législative  Troisième partie : Durée du travail, salaire, intéressement, participation et épargne salariale  Livre Ier : Durée du travail, repos et congés  Titre IV : Congés payés et autres congés  Chapitre II : Autres congés  S

--- Rang 2 | Score BM25 : 10.47 | NON PERTINENT
   LEGIARTI000029217417 LEGI article/LEGI/ARTI/00/00/29/21/74/LEGIARTI000029217417.xml Article   R242-1-1 MODIFIE 2014-07-11 2018-09-30 AUTONOME   Code de la sécurité sociale  Partie réglementaire - Décrets en Conseil d'Etat  Livre II : Organisation du régime général - Action de prévention - Action sanitaire et sociale des caisses  Titre IV : Ressources  Chapitre 2 : Assiette, taux et calcul des c

--- Rang 3 | Score BM25 : 10.47

'\n\n### 2. BM25 avec filtrage métier minimal ("Code du travail")\n\n**Corpus :**\n- 882 documents après filtrage\n\n**Résultats :**\n- Recall@10 = **0,667**\n- MRR = **0,375**\n- nDCG@10 = **0,438**\n\n**Interprétation :**\n- Le filtrage réduit significativement la taille du corpus, mais n’améliore pas encore les métriques globales.\n- Le rappel reste identique : les faux négatifs observés précédemment persistent.\n- La légère baisse de la nDCG indique que certains signaux lexicaux accidentels ont disparu avec le filtrage.\n\n**Analyse qualitative :**\n- Le filtrage par simple sous-chaîne réduit une partie du bruit, mais reste insuffisant.\n- Certains documents hors périmètre apparaissent encore, car ils mentionnent le Code du travail sans en relever réellement.\n- Les erreurs principales ne proviennent donc pas uniquement du bruit inter-code, mais :\n  - de la formulation des requêtes,\n  - de la représentation textuelle des documents,\n  - et des limites intrinsèques du lexical pur.

## ✅ Lecture & interprétation

- 🔁 **Recall@k** : indique si au moins **un** document pertinent apparaît dans le top-*k*.
- 🥇 **MRR** : valorise le fait de trouver **vite** un document pertinent (rang du 1er pertinent).
- 📉 **nDCG@k** : mesure la qualité globale du ranking (pénalise les pertinents trop bas).

👉 Cette étape sert à **quantifier** l’impact du filtrage métier *avant* d’introduire une couche sémantique (embeddings / dense).



### 2. BM25 avec filtrage métier minimal ("Code du travail")

**Corpus :**
- 882 documents après filtrage

**Résultats :**
- Recall@10 = **0,667**
- MRR = **0,375**
- nDCG@10 = **0,438**

**Interprétation :**
- Le filtrage réduit significativement la taille du corpus, mais n’améliore pas encore les métriques globales.
- Le rappel reste identique : les faux négatifs observés précédemment persistent.
- La légère baisse de la nDCG indique que certains signaux lexicaux accidentels ont disparu avec le filtrage.

**Analyse qualitative :**
- Le filtrage par simple sous-chaîne réduit une partie du bruit, mais reste insuffisant.
- Certains documents hors périmètre apparaissent encore, car ils mentionnent le Code du travail sans en relever réellement.
- Les erreurs principales ne proviennent donc pas uniquement du bruit inter-code, mais :
  - de la formulation des requêtes,
  - de la représentation textuelle des documents,
  - et des limites intrinsèques du lexical pur.

---

### Enseignement principal

Le filtrage métier minimal, bien que nécessaire, ne suffit pas à résoudre les faux négatifs ni les problèmes de ranking observés.  
Ces résultats indiquent que les limites actuelles sont principalement **sémantiques**, et non uniquement liées au périmètre documentaire.

## Comparatif synthétique – Impact du filtrage métier sur le retrieval

| Configuration                   | Taille du corpus | Recall@10 | MRR   | nDCG@10 | Lecture principale |
|---------------------------------|------------------|-----------|-------|---------|--------------------|
| BM25 brut                       | 4 422            | 0,667     | 0,375 | 0,459   | Bonne couverture sur requêtes lexicales, bruit élevé |
| BM25 + filtre "Code du travail" | 882              | 0,667     | 0,375 | 0,438   | Bruit réduit mais FN persistants, gain limité |

---

### Conclusion opérationnelle

- Le filtrage métier est une **étape nécessaire**, mais non suffisante.
- Les métriques montrent que :
  - le rappel ne s’améliore pas,
  - le classement reste perfectible,
  - les faux négatifs critiques persistent.
- Les prochaines améliorations doivent cibler :
  - la compréhension sémantique des requêtes,
  - la représentation des documents,
  - puis le ranking fin.

👉 Ces constats justifient l’introduction d’un retrieval sémantique par embeddings, 
tout en conservant le BM25 comme baseline et garde-fou.

